In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
import sys
import io

In [8]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file


In [9]:
from __future__ import print_function



path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

606208/600901 [==============================] - 7s 12us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/60
1565/1565 [==============================] - ETA: 0s - loss: 1.9656
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "g
and straining of a creating, shaping, "
g
and straining of a creating, shaping, and there in the soul and there is a man in the order there in the free of the condition of the condined and there is the conding of the such and in the conding of the still and the free the condined the fore the condined there of the conding of the formetion of the fore there in the senses and the condition of the religious in the still and the order there is possed in the condined to the and the
----- diversity: 0.5
----- Generating with seed: "g
and straining of a creating, shaping, "
g
and straining of a creating, shaping, and sease and in not most of procurst to distincis of such a man--

instances, and be the like it is requice that is as a sears and men that the most conscious and extend of the mutual exertate and conception of the belief and fact and self-sinsing the and promother in the constantice, and of man who
has believed of our pris
----- diversity: 1.0
----- Generating with seed: "he extinction of the belief
that a god g"
he extinction of the belief
that a god god great concerment butlent--elpes, caumangly out
of themself, of the while
good, is, but the lustoge sumental ewold lute "who soqe them still be being and disinties of revenge of desire for him,
and impored mere time to chabitife-best upon psychologist, us's e: in soundood, no peich asoured scitnce of europeans that i the church uses, guature, for undars" so
dus to brange of accorcest sin? "ambun
----- diversity: 1.2
----- Generating with seed: "he extinction of the belief
that a god g"
he extinction of the belief
that a god gn-infection instinct, reoneers at the ietenoference. to this distfultion of 

tuigrain get plense--lust thems conjected mofalled the considering as but evolur of lassiti, as liter him a siff sudden, dy lot, is whil castiverts, but justife and ripwite:--to psychology
kind upon such form along and
prequineful and night, he immediousneness of vain demance worcul
1565/1565 [==============================] - 274s 175ms/step - loss: 1.4135
Epoch 8/60
1565/1565 [==============================] - ETA: 0s - loss: 1.4000
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "een born in our own time it would have b"
een born in our own time it would have been the substitude of the still of the experience to the strongest the former to the substitude of the spirituality of the experience the substitude of the same to the constitute which is the same the philosophy of the standards and the standards and constitute and condition of the soul and conduct of the soul and the sight of the still of the substitude of the soul and and superior, which

h, as a personal consequence brings in is the sublimerance and are only distrust and things. and the struggle of man to the sensual fact, such a desires and called and sensible and soul and desires and excipance of interpretess and more and the subject of europe to consequence of our result and reason to the results and in the action of the dangerous or or "the great who had to the case of the higher, and still naturally to be his depor
----- diversity: 1.0
----- Generating with seed: "h, as a personal consequence brings in i"
h, as a personal consequence brings in itself though it wise: they daf been is ascetical
or nowardedly mane twought with a profound disindeathen taste.mes of their proteser: he domain have
doun tha conscience.

     sestins, for !were latuant, as a particul-fore to mankind for the field that, that that is an advent everything load sunsim stilling of justed is among actss in itself betrays suffermin arproneed as to the jakeble
man a livi
----- diversity: 1.2
-----

prended from the swammer-reveroded," is toward they spetits
and equal millden percoishes! at the god arobithe good by tymightjeestly perfeethe, teace what suepness efflent.

uth when be
aneit
benover-morality is cartain, be laouing" is fable moredr" in colless--his chier
power
his os to do catholity.=--it. ue-rattery and befor
1565/1565 [==============================] - 428s 273ms/step - loss: 1.3520
Epoch 15/60
1565/1565 [==============================] - ETA: 0s - loss: 1.3471
----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "gainst "nature" and also against "reason"
gainst "nature" and also against "reason of the sense of the prompting the sensually in the standard of the sense of the other that the sense of the same man, a philosophy to the sense of the stronger the most procession of the sensually the sense of the strong and something the process of the self-contempt and the sensible the sensible the sensible still not the stronger the strong

...

"neres
flece and caning the effect it , however gives do men from anothing men-aem in let the entitede of should have not only itself.--how respossb: brathime.=--he do an-, not could a rod
and camereancer. how vonvaih this organs bey. centings over saminess, and
1565/1565 [==============================] - 302s 193ms/step - loss: 1.2720
Epoch 46/60
1565/1565 [==============================] - ETA: 0s - loss: 1.2700
----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "man
depth" be what it will--among oursel"
man
depth" be what it will--among ourselves of the sense of the problem, and some conscience of the problem, and the conscience of the world of the contrary, and a man and more power, a philosopher with the subject of the standard of the contradictors of the same and the subject of the contransication of the contrary, and the art of the end as a problem of the former and strength, a senses and spectance of the sense of the subject of th
----- dive

93. the consequently at the strangted the soul and earow of the sensation of the demonstrated to the worst at last the day, regarded and a fling with it at laugher the conscience for the far of religion of the world what all itself as a belief in the form of sympathess and interpretance in the standing himself and period of the fact, to the same sound the order the most all far as it is the philoso
----- diversity: 1.0
----- Generating with seed: "portal of the temple of human
merit!


9"
portal of the temple of human
merit!


95

=haits say are de enough, the lack of destruction of justiceay in his tasks
cruelty of sultable
and sacde, that accord. there
but they make our hand, accord to vivile apires bark comparant of what is to say have
uncertain, that is so diagest
who we "soon only who
knowacitality: for frences hard, which,
firm?--conception of sading when it geance well dreads experiences to assume for other hsism,
----- diversity: 1.2
----- Generating with seed: "portal of the t

terpeonding to immen, feelings, at psets rests of
strongerogems the so.ning at ss in prey bases, that ? our "psecal owne, this expresses render, or immuted uponding e, for early; it turmoract algermatin
for, and
that
syllument--has alary, the most persist
1565/1565 [==============================] - 319s 204ms/step - loss: 1.2629
Epoch 53/60
1565/1565 [==============================] - ETA: 0s - loss: 1.2612
----- Generating text after Epoch: 52
----- diversity: 0.2
----- Generating with seed: "limitation of all
artistic effects by ti"
limitation of all
artistic effects by time to the same thing is always been the same should be the profoundly to be any will and consequently to be extraording the prospucative to the problem of the subjection of the soul of the soul that the soul and the same thing is not the superiority of the soul of the sense and the same sense of the soul of the superiority of the stand to the point of the sense of the consciousness of the soul and
----- diversity: 

and sense of the most age the sublime the higher. in himself at the interceal prouder and proceine of all the
the sublime that
can not what is only the same sin and longer as it is this conscious and the former and life and the common that is to be precent the same passion an error when it is among happens, and of the refined to the actual case being that is acts which is only the placed devise that the 
----- diversity: 1.0
----- Generating with seed: "thstanding that the finest hands
and sen"
thstanding that the finest hands
and senting to recolly
sest in, are not
a future in help of all hypicring for
is not a
rtbvermly "for advanced," just besome account exagge events"y--ha say the same spirit which for eurogawied, han topor of wspective incalm of traisinged the womans profoutorious refined in carrys of dangerous what is made back as
to retered his values.

,ëunds that fasthing certain winntly remains that a new ages a reas
----- diversity: 1.2
----- Generating with seed: "thstandin

      during self-desilt--it morally evidence: for one worls--that would least in very diberal necessity--he free sever! it had egows, nceations are not need by his ultifici hars rayes co
super-hatrqwours, who who greyourous truth
the strange and their fle sidefictions. pomihs to animal,--we
have had less lusiclarih sympabeor mynally chainta
1565/1565 [==============================] - 215s 137ms/step - loss: 1.2530
Epoch 60/60
1565/1565 [==============================] - ETA: 0s - loss: 1.2566
----- Generating text after Epoch: 59
----- diversity: 0.2
----- Generating with seed: " by his passions and vices, longs to fin"
 by his passions and vices, longs to find the extends and in the prosiher and consequences and sense, and the problems of the subjection of the world and the most and the surmang and the sense, and the sense, and the condition of the sentiment and the contradiction of the contradiction of the spirit to the more and a surmang of the conceptions and precisely the conseq